# Financial Modeling Prep (FMP) API - Stock Analysis Notebook

This notebook demonstrates comprehensive stock analysis using the Financial Modeling Prep API.

## Features Covered:
- Real-time stock quotes and market data
- Company fundamentals and financial statements
- Historical price analysis and technical indicators
- Portfolio analysis and risk assessment
- Stock screening and comparison
- Data visualization and charting

## Prerequisites:
- FMP API key (sign up at https://financialmodelingprep.com/)
- Required packages: pandas, numpy, matplotlib, seaborn, requests

In [ ]:
# Import required libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

# Add parent directory to path
sys.path.append(os.path.dirname(os.getcwd()))

# Import our FMP client and utilities
from client.fmp_client import FMPClient, FMPAPIError
from utils.helpers import (
    format_currency, format_percentage, calculate_returns,
    calculate_volatility, calculate_sharpe_ratio, calculate_beta
)
from utils.config import Config

print("📊 FMP Stock Analysis Notebook Ready!")
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Setup and Configuration

First, let's set up our FMP API client with the API key.

In [ ]:
# Configuration
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
API_KEY = os.getenv('FMP_API_KEY')

if not API_KEY:
    print("❌ Error: FMP_API_KEY not found in environment variables")
    print("Please create a .env file with your API key:")
    print("FMP_API_KEY=your_api_key_here")
    raise ValueError("Missing FMP_API_KEY environment variable")

# Initialize FMP client
client = FMPClient(API_KEY)

# Validate API key
if client.validate_api_key():
    print("✅ API key is valid and ready to use!")
    print(f"🔑 Using API key: {API_KEY[:8]}...{API_KEY[-4:]}")  # Show partial key for verification
else:
    print("❌ Invalid API key. Please check your credentials.")

## 2. Real-time Stock Quote Analysis

Let's start by analyzing real-time data for a specific stock.

In [ ]:
# Stock symbol to analyze
SYMBOL = "AAPL"  # Change this to any stock symbol

# Get real-time quote
quote = client.get_quote(SYMBOL)

if quote:
    print(f"📈 {SYMBOL} - Real-time Quote Analysis")
    print("=" * 40)
    
    # Basic quote data
    current_price = quote['price']
    daily_change = quote['change']
    daily_change_pct = quote['changesPercentage']
    
    print(f"Current Price: {format_currency(current_price)}")
    print(f"Daily Change: {format_currency(daily_change)} ({format_percentage(daily_change_pct/100)})")
    print(f"Day Range: {format_currency(quote['dayLow'])} - {format_currency(quote['dayHigh'])}")
    print(f"52-Week Range: {format_currency(quote['yearLow'])} - {format_currency(quote['yearHigh'])}")
    print(f"Market Cap: {format_currency(quote['marketCap'])}")
    print(f"Volume: {quote['volume']:,}")
    print(f"P/E Ratio: {quote['pe']:.2f}")
    print(f"EPS: {format_currency(quote['eps'])}")
    
    # Store quote data for later use
    quote_data = quote
else:
    print(f"❌ Could not retrieve quote for {SYMBOL}")

## 3. Company Profile and Fundamentals

In [ ]:
# Get company profile
profile = client.get_company_profile(SYMBOL)

if profile and profile[0]:
    p = profile[0]
    
    print(f"🏢 {SYMBOL} - Company Profile")
    print("=" * 30)
    
    print(f"Company Name: {p['companyName']}")
    print(f"Industry: {p['industry']}")
    print(f"Sector: {p['sector']}")
    print(f"Country: {p['country']}")
    print(f"Exchange: {p['exchangeShortName']}")
    print(f"Employees: {p.get('fullTimeEmployees', 'N/A'):,}" if isinstance(p.get('fullTimeEmployees'), (int, float)) else f"Employees: {p.get('fullTimeEmployees', 'N/A')}")
    print(f"Website: {p['website']}")
    print(f"CEO: {p.get('ceo', 'N/A')}")
    
    print(f"\nDescription:\n{p['description'][:500]}...")
    
    # Store profile data
    profile_data = p

## 4. Historical Price Analysis and Visualization

In [ ]:
# Get historical price data (1 year)
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

print(f"📊 Fetching historical data for {SYMBOL} ({start_date} to {end_date})")

historical = client.get_historical_prices(SYMBOL, start_date, end_date)

if historical and 'historical' in historical:
    # Convert to DataFrame
    df = pd.DataFrame(historical['historical'])
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    
    print(f"✅ Retrieved {len(df)} days of historical data")
    
    # Calculate technical indicators
    df['returns'] = df['close'].pct_change()
    df['volatility_30d'] = df['returns'].rolling(30).std() * np.sqrt(252)  # Annualized
    df['sma_20'] = df['close'].rolling(20).mean()  # 20-day moving average
    df['sma_50'] = df['close'].rolling(50).mean()  # 50-day moving average
    
    # Display summary statistics
    print("\n📈 Historical Price Summary:")
    print(f"Period: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
    print(f"Starting Price: {format_currency(df['close'].iloc[0])}")
    print(f"Ending Price: {format_currency(df['close'].iloc[-1])}")
    
    year_return = (df['close'].iloc[-1] - df['close'].iloc[0]) / df['close'].iloc[0]
    print(f"1-Year Return: {format_percentage(year_return)}")
    
    annual_volatility = df['returns'].std() * np.sqrt(252)
    print(f"Annual Volatility: {format_percentage(annual_volatility)}")
    
    print(f"Average Volume: {df['volume'].mean():,.0f}")
    print(f"Highest Price: {format_currency(df['high'].max())}")
    print(f"Lowest Price: {format_currency(df['low'].min())}")
    
    # Store DataFrame for plotting
    historical_df = df
else:
    print(f"❌ Could not retrieve historical data for {SYMBOL}")

In [ ]:
# Create comprehensive price chart
if 'historical_df' in locals():
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # Price chart with moving averages
    axes[0].plot(historical_df['date'], historical_df['close'], label='Close Price', linewidth=2)
    axes[0].plot(historical_df['date'], historical_df['sma_20'], label='20-day SMA', alpha=0.7)
    axes[0].plot(historical_df['date'], historical_df['sma_50'], label='50-day SMA', alpha=0.7)
    axes[0].set_title(f'{SYMBOL} - Price Chart with Moving Averages', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Price ($)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Volume chart
    axes[1].bar(historical_df['date'], historical_df['volume'], alpha=0.6, color='orange')
    axes[1].set_title(f'{SYMBOL} - Trading Volume', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Volume')
    axes[1].grid(True, alpha=0.3)
    
    # Returns distribution
    axes[2].hist(historical_df['returns'].dropna(), bins=50, alpha=0.7, color='green', edgecolor='black')
    axes[2].axvline(historical_df['returns'].mean(), color='red', linestyle='--', label=f'Mean: {format_percentage(historical_df["returns"].mean())}')
    axes[2].set_title(f'{SYMBOL} - Daily Returns Distribution', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('Daily Returns')
    axes[2].set_ylabel('Frequency')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 5. Financial Statements Analysis

In [ ]:
# Get financial statements (last 5 years)
print(f"💰 {SYMBOL} - Financial Statements Analysis")
print("=" * 45)

# Income Statement
income = client.get_income_statement(SYMBOL, limit=5)
balance = client.get_balance_sheet(SYMBOL, limit=5)
cashflow = client.get_cash_flow_statement(SYMBOL, limit=5)

if income:
    # Convert to DataFrame for easier analysis
    income_df = pd.DataFrame(income)
    income_df['date'] = pd.to_datetime(income_df['date'])
    income_df = income_df.sort_values('date')
    
    print("📊 Income Statement Trends (Last 5 Years):")
    print(income_df[['date', 'revenue', 'grossProfit', 'operatingIncome', 'netIncome']].to_string(index=False))
    
    # Calculate growth rates
    if len(income_df) > 1:
        revenue_growth = (income_df['revenue'].iloc[-1] / income_df['revenue'].iloc[0]) ** (1/(len(income_df)-1)) - 1
        net_income_growth = (income_df['netIncome'].iloc[-1] / income_df['netIncome'].iloc[0]) ** (1/(len(income_df)-1)) - 1
        
        print(f"\n📈 Growth Metrics:")
        print(f"Revenue CAGR: {format_percentage(revenue_growth)}")
        print(f"Net Income CAGR: {format_percentage(net_income_growth)}")
    
    # Store for visualization
    financial_data = {
        'income': income_df,
        'balance': pd.DataFrame(balance) if balance else None,
        'cashflow': pd.DataFrame(cashflow) if cashflow else None
    }

In [ ]:
# Visualize financial trends
if 'financial_data' in locals() and financial_data['income'] is not None:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    df = financial_data['income']
    
    # Revenue trend
    axes[0,0].plot(df['date'], df['revenue'], marker='o', linewidth=2, markersize=6)
    axes[0,0].set_title(f'{SYMBOL} - Revenue Trend', fontweight='bold')
    axes[0,0].set_ylabel('Revenue ($)')
    axes[0,0].grid(True, alpha=0.3)
    
    # Net Income trend
    axes[0,1].plot(df['date'], df['netIncome'], marker='o', linewidth=2, markersize=6, color='green')
    axes[0,1].set_title(f'{SYMBOL} - Net Income Trend', fontweight='bold')
    axes[0,1].set_ylabel('Net Income ($)')
    axes[0,1].grid(True, alpha=0.3)
    
    # Profit margins
    df['gross_margin'] = df['grossProfit'] / df['revenue']
    df['operating_margin'] = df['operatingIncome'] / df['revenue']
    df['net_margin'] = df['netIncome'] / df['revenue']
    
    axes[1,0].plot(df['date'], df['gross_margin'], marker='o', label='Gross Margin')
    axes[1,0].plot(df['date'], df['operating_margin'], marker='s', label='Operating Margin')
    axes[1,0].plot(df['date'], df['net_margin'], marker='^', label='Net Margin')
    axes[1,0].set_title(f'{SYMBOL} - Profit Margins', fontweight='bold')
    axes[1,0].set_ylabel('Margin (%)')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # EPS trend
    axes[1,1].plot(df['date'], df['eps'], marker='o', linewidth=2, markersize=6, color='purple')
    axes[1,1].set_title(f'{SYMBOL} - Earnings Per Share (EPS)', fontweight='bold')
    axes[1,1].set_ylabel('EPS ($)')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 6. Portfolio Analysis

Let's create a sample portfolio and analyze its performance and risk characteristics.

In [ ]:
# Define a sample portfolio
portfolio = {
    'AAPL': {'shares': 100, 'purchase_price': 180.00},
    'MSFT': {'shares': 50, 'purchase_price': 350.00},
    'GOOGL': {'shares': 30, 'purchase_price': 140.00},
    'AMZN': {'shares': 25, 'purchase_price': 145.00},
    'TSLA': {'shares': 20, 'purchase_price': 200.00}
}

print("📊 Portfolio Analysis")
print("=" * 25)

# Get current quotes for all holdings
symbols = list(portfolio.keys())
quotes = client.get_quotes(symbols)

if quotes:
    # Calculate portfolio metrics
    portfolio_data = []
    total_value = 0
    total_cost = 0
    
    for quote in quotes:
        symbol = quote['symbol']
        if symbol in portfolio:
            shares = portfolio[symbol]['shares']
            purchase_price = portfolio[symbol]['purchase_price']
            current_price = quote['price']
            
            current_value = shares * current_price
            cost_basis = shares * purchase_price
            gain_loss = current_value - cost_basis
            return_pct = (current_price - purchase_price) / purchase_price
            
            portfolio_data.append({
                'Symbol': symbol,
                'Shares': shares,
                'Purchase Price': purchase_price,
                'Current Price': current_price,
                'Current Value': current_value,
                'Cost Basis': cost_basis,
                'Gain/Loss': gain_loss,
                'Return %': return_pct
            })
            
            total_value += current_value
            total_cost += cost_basis
    
    # Create portfolio DataFrame
    portfolio_df = pd.DataFrame(portfolio_data)
    portfolio_df['Weight %'] = portfolio_df['Current Value'] / total_value
    
    print("📈 Portfolio Holdings:")
    print(portfolio_df.round(2).to_string(index=False))
    
    print(f"\n💰 Portfolio Summary:")
    print(f"Total Value: {format_currency(total_value)}")
    print(f"Total Cost: {format_currency(total_cost)}")
    print(f"Total Gain/Loss: {format_currency(total_value - total_cost)}")
    print(f"Total Return: {format_percentage((total_value - total_cost) / total_cost)}")
    
    # Store for visualization
    portfolio_analysis = portfolio_df

In [ ]:
# Portfolio visualization
if 'portfolio_analysis' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Portfolio allocation pie chart
    axes[0,0].pie(portfolio_analysis['Current Value'], labels=portfolio_analysis['Symbol'], 
                  autopct='%1.1f%%', startangle=90)
    axes[0,0].set_title('Portfolio Allocation by Value', fontweight='bold')
    
    # Returns by holding
    colors = ['green' if x > 0 else 'red' for x in portfolio_analysis['Return %']]
    axes[0,1].bar(portfolio_analysis['Symbol'], portfolio_analysis['Return %'], color=colors)
    axes[0,1].set_title('Individual Stock Returns', fontweight='bold')
    axes[0,1].set_ylabel('Return (%)')
    axes[0,1].grid(True, alpha=0.3)
    
    # Gain/Loss by holding
    colors = ['green' if x > 0 else 'red' for x in portfolio_analysis['Gain/Loss']]
    axes[1,0].bar(portfolio_analysis['Symbol'], portfolio_analysis['Gain/Loss'], color=colors)
    axes[1,0].set_title('Gain/Loss by Holding ($)', fontweight='bold')
    axes[1,0].set_ylabel('Gain/Loss ($)')
    axes[1,0].grid(True, alpha=0.3)
    
    # Current value by holding
    axes[1,1].bar(portfolio_analysis['Symbol'], portfolio_analysis['Current Value'], 
                  color='blue', alpha=0.7)
    axes[1,1].set_title('Current Value by Holding', fontweight='bold')
    axes[1,1].set_ylabel('Value ($)')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 7. Stock Screening and Comparison

Let's screen stocks based on specific criteria and compare multiple stocks.

In [ ]:
# Compare tech stocks
tech_stocks = ['AAPL', 'MSFT', 'GOOGL', 'META', 'AMZN', 'TSLA', 'NVDA', 'NFLX']

print("🔍 Tech Stock Comparison")
print("=" * 25)

tech_quotes = client.get_quotes(tech_stocks)

if tech_quotes:
    comparison_data = []
    
    for quote in tech_quotes:
        symbol = quote['symbol']
        comparison_data.append({
            'Symbol': symbol,
            'Price': quote['price'],
            'Market Cap': quote.get('marketCap', 0),
            'P/E Ratio': quote.get('pe', 0),
            'Daily Change %': quote.get('changesPercentage', 0),
            'Volume': quote.get('volume', 0),
            'EPS': quote.get('eps', 0)
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Sort by market cap
    comparison_df = comparison_df.sort_values('Market Cap', ascending=False)
    
    print("📊 Tech Stock Metrics:")
    print(comparison_df.round(2).to_string(index=False))
    
    # Store for visualization
    tech_comparison = comparison_df

In [ ]:
# Visualize tech stock comparison
if 'tech_comparison' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Market cap comparison
    axes[0,0].bar(tech_comparison['Symbol'], tech_comparison['Market Cap'], color='skyblue')
    axes[0,0].set_title('Market Capitalization Comparison', fontweight='bold')
    axes[0,0].set_ylabel('Market Cap ($)')
    axes[0,0].tick_params(axis='x', rotation=45)
    axes[0,0].grid(True, alpha=0.3)
    
    # P/E ratio comparison
    pe_data = tech_comparison[tech_comparison['P/E Ratio'] > 0]  # Filter out negative P/E
    axes[0,1].bar(pe_data['Symbol'], pe_data['P/E Ratio'], color='lightgreen')
    axes[0,1].set_title('P/E Ratio Comparison', fontweight='bold')
    axes[0,1].set_ylabel('P/E Ratio')
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].grid(True, alpha=0.3)
    
    # Daily performance
    colors = ['green' if x > 0 else 'red' for x in tech_comparison['Daily Change %']]
    axes[1,0].bar(tech_comparison['Symbol'], tech_comparison['Daily Change %'], color=colors)
    axes[1,0].set_title('Daily Performance (%)', fontweight='bold')
    axes[1,0].set_ylabel('Daily Change (%)')
    axes[1,0].tick_params(axis='x', rotation=45)
    axes[1,0].grid(True, alpha=0.3)
    
    # Price comparison
    axes[1,1].bar(tech_comparison['Symbol'], tech_comparison['Price'], color='orange')
    axes[1,1].set_title('Stock Price Comparison', fontweight='bold')
    axes[1,1].set_ylabel('Price ($)')
    axes[1,1].tick_params(axis='x', rotation=45)
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 8. Market Analysis and Sector Performance

In [ ]:
# Analyze sector performance
print("🌍 Market and Sector Analysis")
print("=" * 30)

# Get sector performance
sectors = client.get_sector_performance()

if sectors:
    sector_df = pd.DataFrame(sectors)
    sector_df['changesPercentage'] = pd.to_numeric(sector_df['changesPercentage'], errors='coerce')
    sector_df = sector_df.dropna(subset=['changesPercentage'])
    sector_df = sector_df.sort_values('changesPercentage', ascending=False)
    
    print("📊 Sector Performance:")
    for _, row in sector_df.head(10).iterrows():
        sector_name = row['sector']
        performance = row['changesPercentage']
        print(f"  {sector_name}: {format_percentage(performance/100)}")
    
    # Store for visualization
    sector_performance = sector_df

In [ ]:
# Visualize sector performance
if 'sector_performance' in locals():
    plt.figure(figsize=(12, 8))
    
    # Create color map based on performance
    colors = ['green' if x > 0 else 'red' for x in sector_performance['changesPercentage']]
    
    plt.barh(range(len(sector_performance)), sector_performance['changesPercentage'], color=colors)
    plt.yticks(range(len(sector_performance)), sector_performance['sector'])
    plt.xlabel('Performance (%)')
    plt.title('Sector Performance Overview', fontsize=16, fontweight='bold')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.5)
    plt.grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for i, v in enumerate(sector_performance['changesPercentage']):
        plt.text(v + 0.01 if v > 0 else v - 0.01, i, f'{v:.2f}%', 
                va='center', ha='left' if v > 0 else 'right')
    
    plt.tight_layout()
    plt.show()

## 9. Risk Analysis and Correlation

In [ ]:
# Risk analysis for our portfolio stocks
print("⚠️ Risk Analysis")
print("=" * 15)

# Get historical data for correlation analysis
symbols_for_risk = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'SPY']  # Include SPY as market benchmark
risk_data = {}

end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=252)).strftime('%Y-%m-%d')  # 1 year

for symbol in symbols_for_risk:
    historical = client.get_historical_prices(symbol, start_date, end_date)
    if historical and 'historical' in historical:
        df = pd.DataFrame(historical['historical'])
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date')
        df['returns'] = df['close'].pct_change()
        risk_data[symbol] = df[['date', 'close', 'returns']].dropna()

if risk_data:
    # Create returns matrix for correlation analysis
    returns_matrix = pd.DataFrame()
    for symbol, data in risk_data.items():
        returns_matrix[symbol] = data['returns'].values
    
    # Calculate correlation matrix
    correlation_matrix = returns_matrix.corr()
    
    print("📊 Correlation Matrix:")
    print(correlation_matrix.round(3).to_string())
    
    # Calculate individual stock metrics vs SPY
    spy_returns = risk_data['SPY']['returns'].values
    
    print("\n📈 Risk Metrics vs SPY:")
    for symbol in symbols_for_risk[:-1]:  # Exclude SPY itself
        if symbol in risk_data:
            stock_returns = risk_data[symbol]['returns'].values
            
            # Calculate beta
            if len(stock_returns) == len(spy_returns):
                beta = calculate_beta(stock_returns, spy_returns)
                volatility = calculate_volatility(stock_returns)
                
                print(f"  {symbol}: Beta = {beta:.2f}, Volatility = {format_percentage(volatility)}")
    
    # Store for visualization
    correlation_data = correlation_matrix

In [ ]:
# Visualize correlation matrix
if 'correlation_data' in locals():
    plt.figure(figsize=(10, 8))
    
    # Create heatmap
    mask = np.triu(np.ones_like(correlation_data, dtype=bool))  # Mask upper triangle
    sns.heatmap(correlation_data, mask=mask, annot=True, cmap='RdYlBu', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": .8})
    
    plt.title('Stock Correlation Matrix', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 10. Summary and Insights

Let's create a summary of our analysis with key insights and recommendations.

In [ ]:
# Generate analysis summary
print("📋 ANALYSIS SUMMARY")
print("=" * 20)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# Stock Analysis Summary
if 'quote_data' in locals():
    print(f"🔍 PRIMARY STOCK ANALYSIS: {SYMBOL}")
    print("-" * 35)
    print(f"Current Price: {format_currency(quote_data['price'])}")
    print(f"Daily Change: {format_percentage(quote_data['changesPercentage']/100)}")
    print(f"Market Cap: {format_currency(quote_data['marketCap'])}")
    print(f"P/E Ratio: {quote_data['pe']:.2f}")
    
    # Simple valuation assessment
    pe_ratio = quote_data['pe']
    if pe_ratio < 15:
        valuation = "Potentially Undervalued"
    elif pe_ratio > 30:
        valuation = "Potentially Overvalued"
    else:
        valuation = "Fairly Valued"
    print(f"Valuation Assessment: {valuation}")
    print()

# Portfolio Summary
if 'portfolio_analysis' in locals():
    print("💰 PORTFOLIO SUMMARY")
    print("-" * 18)
    total_value = portfolio_analysis['Current Value'].sum()
    total_cost = portfolio_analysis['Cost Basis'].sum()
    portfolio_return = (total_value - total_cost) / total_cost
    
    print(f"Total Portfolio Value: {format_currency(total_value)}")
    print(f"Total Return: {format_percentage(portfolio_return)}")
    
    # Best and worst performers
    best_performer = portfolio_analysis.loc[portfolio_analysis['Return %'].idxmax()]
    worst_performer = portfolio_analysis.loc[portfolio_analysis['Return %'].idxmin()]
    
    print(f"Best Performer: {best_performer['Symbol']} ({format_percentage(best_performer['Return %'])})")
    print(f"Worst Performer: {worst_performer['Symbol']} ({format_percentage(worst_performer['Return %'])})")
    print()

# Market Context
if 'sector_performance' in locals():
    print("🌍 MARKET CONTEXT")
    print("-" * 15)
    best_sector = sector_performance.iloc[0]
    worst_sector = sector_performance.iloc[-1]
    
    print(f"Best Performing Sector: {best_sector['sector']} ({format_percentage(best_sector['changesPercentage']/100)})")
    print(f"Worst Performing Sector: {worst_sector['sector']} ({format_percentage(worst_sector['changesPercentage']/100)})")
    print()

# Key Insights
print("💡 KEY INSIGHTS")
print("-" * 13)
print("✅ Completed comprehensive financial analysis using FMP API")
print("📊 Analyzed real-time quotes, historical data, and financial statements")
print("📈 Performed portfolio analysis with risk assessment")
print("🔍 Conducted stock screening and sector comparison")
print("⚠️  Calculated correlation and risk metrics")
print()
print("📋 RECOMMENDATIONS:")
print("• Continue monitoring portfolio performance and rebalance as needed")
print("• Consider diversification across different sectors and asset classes")
print("• Regular review of individual stock fundamentals and valuations")
print("• Stay informed about market trends and economic indicators")
print()
print("⚠️  DISCLAIMER: This analysis is for educational purposes only.")
print("Always consult with a qualified financial advisor before making investment decisions.")

## Next Steps

This notebook has demonstrated the key capabilities of the FMP API for financial analysis. Here are some ideas for extending this analysis:

1. **Advanced Technical Analysis**: Add more technical indicators like RSI, MACD, Bollinger Bands
2. **Fundamental Analysis**: Deep dive into financial ratios, DCF modeling, and peer comparisons
3. **Risk Management**: Implement Value at Risk (VaR) calculations and Monte Carlo simulations
4. **Automated Alerts**: Set up monitoring for price movements, earnings announcements, etc.
5. **Machine Learning**: Apply ML models for price prediction and pattern recognition
6. **Real-time Dashboard**: Create interactive dashboards using tools like Plotly or Streamlit

Feel free to modify and extend this notebook for your specific analysis needs!